# Import Libraries

In [1]:
import sys
sys.path.append("..")
import seaborn as sb
import dgl
import torch

import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from tqdm import tqdm
import torch.nn.functional as F

import statistics
import argparse
import random

from dgl.data import CitationGraphDataset
# from dgl.nn import GINConv, GraphConv, SAGEConv
import seaborn as sb
import torch
import torch.nn as nn
from torch.autograd import Variable
from scipy.optimize import linear_sum_assignment
import scipy
import scipy.optimize
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import sklearn as sk
import networkx as nx


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import random
import math
import time


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data

from torch_geometric.nn import GCNConv,GINConv,SAGEConv,GATConv,PNAConv, GraphSAGE
from torch_geometric.utils import add_self_loops
from torch_geometric.transforms import normalize_features
from pygod.utils import load_data
from pygod.utils.utility import check_parameter
from pygod.metrics import eval_roc_auc
from pygod.generator import gen_contextual_outliers, gen_structural_outliers
from scipy.linalg import sqrtm

/home/koo/miniconda3/envs/gadnr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

# Utils

In [9]:

def _normalize(x):
    x_min = x.min()
    x_max = x.max()
    x_norm = (x - x_min)/x_max
    return x_norm

def gen_joint_structural_outliers(data, m, n, random_state=None):
    """
    We randomly select n nodes from the network which will be the anomalies 
    and for each node we select m nodes from the network. 
    We connect each of n nodes with the m other nodes.

    Parameters
    ----------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The input data.
    m : int
        Number nodes in the outlier cliques.
    n : int
        Number of outlier cliques.
    p : int, optional
        Probability of edge drop in cliques. Default: ``0``.
    random_state : int, optional
        The seed to control the randomness, Default: ``None``.

    Returns
    -------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The structural outlier graph with injected edges.
    y_outlier : torch.Tensor
        The outlier label tensor where 1 represents outliers and 0 represents
        regular nodes.
    """

    if not isinstance(data, Data):
        raise TypeError("data should be torch_geometric.data.Data")

    if isinstance(m, int):
        check_parameter(m, low=0, high=data.num_nodes, param_name='m')
    else:
        raise ValueError("m should be int, got %s" % m)

    if isinstance(n, int):
        check_parameter(n, low=0, high=data.num_nodes, param_name='n')
    else:
        raise ValueError("n should be int, got %s" % n)

    check_parameter(m * n, low=0, high=data.num_nodes, param_name='m*n')

    if random_state:
        np.random.seed(random_state)


    outlier_idx = np.random.choice(data.num_nodes, size=n, replace=False)
    all_nodes = [i for i in range(data.num_nodes)]
    rem_nodes = []
    
    for node in all_nodes:
        if node is not outlier_idx:
            rem_nodes.append(node)
    
    
    
    new_edges = []
    
    # connect all m nodes in each clique
    for i in range(0, n):
        other_idx = np.random.choice(data.num_nodes, size=m, replace=False)
        for j in other_idx:
            new_edges.append(torch.tensor([[i, j]], dtype=torch.long))
                    

    new_edges = torch.cat(new_edges)


    # y_outlier[outlier_idx] = 1
    outlier_idx = torch.tensor(outlier_idx, dtype=torch.long)
    y_outlier = torch.zeros(data.x.shape[0], dtype=torch.long)
    y_outlier[outlier_idx] = 1

    data.edge_index = torch.cat([data.edge_index, new_edges.T], dim=1)

    return data, y_outlier


def KL_neighbor_loss(predictions, targets, mask_len):
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)
    
    nn = x1.shape[0]
    h_dim = x1.shape[1]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / max((nn-1),1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / max((nn-1),1)
    
    eye = torch.eye(h_dim)
    cov_x1 = cov_x1 + eye
    cov_x2 = cov_x2 + eye
    
    KL_loss = 0.5 * (math.log(torch.det(cov_x1) / torch.det(cov_x2)) - h_dim  + torch.trace(torch.inverse(cov_x2).matmul(cov_x1)) 
            + (mean_x2 - mean_x1).reshape(1,-1).matmul(torch.inverse(cov_x2)).matmul(mean_x2 - mean_x1))
    KL_loss = KL_loss.to(device)
    return KL_loss

def W2_neighbor_loss(predictions, targets, mask_len):
    
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)

    nn = x1.shape[0]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / (nn-1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / (nn-1)
    

    W2_loss = torch.square(mean_x1-mean_x2).sum() + torch.trace(cov_x1 + cov_x2 
                     + 2 * sqrtm(sqrtm(cov_x1) @ (cov_x2.numpy()) @ (sqrtm(cov_x1))))

    return W2_loss



# Layers

In [10]:
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        

        super(MLP, self).__init__()

        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = self.linears[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 0, 1)
                    h = torch.transpose(h, 1, 2)
                    
                h = self.batch_norms[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 1, 2)
                    h = torch.transpose(h, 0, 1)

                h = F.relu(h)
                # h = F.relu(self.linears[layer](h))
                
            return self.linears[self.num_layers - 1](h)


class MLP_generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_generator, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(output_dim, output_dim)
        self.linear3 = nn.Linear(output_dim, output_dim)
        self.linear4 = nn.Linear(output_dim, output_dim)

    def forward(self, embedding):
        neighbor_embedding = F.relu(self.linear(embedding))
        neighbor_embedding = F.relu(self.linear2(neighbor_embedding))
        neighbor_embedding = F.relu(self.linear3(neighbor_embedding))
        neighbor_embedding = self.linear4(neighbor_embedding)
        return neighbor_embedding


class PairNorm(nn.Module):
    def __init__(self, mode='PN', scale=10):
        
        assert mode in ['None', 'PN', 'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

        # Scale can be set based on origina data, and also the current feature lengths.
        # We leave the experiments to future. A good pool we used for choosing scale:
        # [0.1, 1, 10, 50, 100]
    def forward(self, x):
        if self.mode == 'None':
            return x
        col_mean = x.mean(dim=0)
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt()
            x = self.scale * x / rownorm_mean
        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual
        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean
        return x


# FNN
class FNN(nn.Module):
    def __init__(self, in_features, hidden, out_features, layer_num):
        super(FNN, self).__init__()
        self.linear1 = MLP(layer_num, in_features, hidden, out_features)
        self.linear2 = nn.Linear(out_features, out_features)
    def forward(self, embedding):
        x = self.linear1(embedding)
        x = self.linear2(F.relu(x))
        x = F.relu(x)
        return x


# GAD-NR

In [14]:

# Training
def train(data, y, yc, ys, yj, ysj, lr, epoch, device, encoder, lambda_loss1, lambda_loss2, lambda_loss3, hidden_dim, sample_size=10,loss_step=20,
          real_loss=False,calculate_contextual=False,calculate_structural=False):
    '''
     Main training function
     INPUT:
     -----------------------
     data : torch geometric dataset object
     lr    :    learning rate
     epoch     :    number of training epoch
     device     :   CPU or GPU
     encoder    :    GCN or GIN or GraphSAGE
     lambda_loss    :   Trade-off between degree loss and neighborhood reconstruction loss
     hidden_dim     :   latent variable dimension
    '''
    
    in_nodes = data.edge_index[0,:]
    out_nodes = data.edge_index[1,:]
    
    
    neighbor_dict = {}
    for in_node, out_node in zip(in_nodes, out_nodes):
        if in_node.item() not in neighbor_dict:
            neighbor_dict[in_node.item()] = []
        neighbor_dict[in_node.item()].append(out_node.item())

    neighbor_num_list = []
    for i in neighbor_dict:
        neighbor_num_list.append(len(neighbor_dict[i]))
    
    neighbor_num_list = torch.tensor(neighbor_num_list).to(device)
    
    in_dim = data.x.shape[1]
    GNNModel = GNNStructEncoder(in_dim, hidden_dim, hidden_dim, 2, sample_size, device=device, 
                    neighbor_num_list=neighbor_num_list, GNN_name=encoder, 
                    lambda_loss1=lambda_loss1, lambda_loss2=lambda_loss2,lambda_loss3=lambda_loss3)
    GNNModel.to(device)
    degree_params = list(map(id, GNNModel.degree_decoder.parameters()))
    base_params = filter(lambda p: id(p) not in degree_params,
                         GNNModel.parameters())

    opt = torch.optim.Adam([{'params': base_params}, {'params': GNNModel.degree_decoder.parameters(), 'lr': 1e-2}],lr=lr, weight_decay=0.0003)
    min_loss = float('inf')
    arg_min_loss_per_node = None
    
    best_auc = 0
    best_auc_contextual = 0
    best_auc_dense_structural = 0
    best_auc_joint_structural = 0
    best_auc_structure_type = 0
    
        
    loss_values = []
    for i in tqdm(range(1,epoch+1,1)):
        
        # st = time.time()
        
        if i%loss_step==0:
            GNNModel.lambda_loss2 = GNNModel.lambda_loss2 + 0.5
            GNNModel.lambda_loss3 = GNNModel.lambda_loss3 / 2
        
        loss,loss_per_node,h_loss,degree_loss,feature_loss = GNNModel(data.edge_index, data.x, neighbor_num_list, neighbor_dict, device=device)
        
        
        
        loss_per_node = loss_per_node.cpu().detach()
        
        h_loss = h_loss.cpu().detach()
        degree_loss = degree_loss.cpu().detach()
        feature_loss = feature_loss.cpu().detach()
        
        h_loss_norm = h_loss / (torch.max(h_loss) - torch.min(h_loss))
        degree_loss_norm = degree_loss / (torch.max(degree_loss) - torch.min(degree_loss))
        feature_loss_norm = feature_loss / (torch.max(feature_loss) - torch.min(feature_loss))
        
        comb_loss = args.h_loss_weight * h_loss_norm + args.degree_loss_weight *  degree_loss_norm + args.feature_loss_weight * feature_loss_norm
        
        if real_loss:
            comp_loss = loss_per_node
        else:
            comp_loss = comb_loss
            
        
        auc_score = eval_roc_auc(y.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score(benchmark/combined): ", auc_score)
        
        contextual_auc_score = eval_roc_auc(yc.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (contextual): ", contextual_auc_score)

        dense_structural_auc_score = eval_roc_auc(ys.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structural): ", dense_structural_auc_score)
        
        joint_type_auc_score = eval_roc_auc(yj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (joint-type): ", joint_type_auc_score)
        
        structure_type_auc_score = eval_roc_auc(ysj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structure type): ", structure_type_auc_score) 
        
        best_auc = max(best_auc, auc_score)
        best_auc_contextual = max(best_auc_contextual, contextual_auc_score)
        best_auc_dense_structural = max(best_auc_dense_structural, dense_structural_auc_score)
        best_auc_joint_type = max(best_auc_joint_structural, joint_type_auc_score)
        best_auc_structure_type = max(best_auc_structure_type, structure_type_auc_score)
        
        
        
        print("===========================================================================================")
        print("Dataset Name: ",dataset_str, " Best AUC Score(benchmark/combined): ", best_auc)
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (contextual): ", best_auc_contextual)

        
        print("Dataset Name: ",dataset_str, " Best AUC Score (structural): ", best_auc_dense_structural)
        
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (joint-type): ", best_auc_joint_type)
        
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (structure type): ", best_auc_structure_type) 
        print("===========================================================================================")
        
        
        if loss < min_loss:
            min_loss = loss
            arg_min_loss_per_node = loss_per_node
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        loss = loss.cpu().detach()
        loss_values.append(loss)
        
        en = time.time()
        # print("Required Time: ", (en-st)*1000)
        # import sys
        # sys.exit(0)
        
        if args.plot_loss:

            plt.plot(np.array(loss_values), 'r')
            plt.show()
    
    return min_loss.item(), arg_min_loss_per_node.cpu().detach()




def evaluate(model, embeddings, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(embeddings)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train_real_datasets(dataset_str, epoch_num = 10, lr = 5e-6, encoder = "GCN", 
                        lambda_loss1=1e-2, lambda_loss2=1e-3, lambda_loss3=1e-3, sample_size=8, loss_step=20, hidden_dim=None,
                        real_loss=False,calculate_contextual=False,calculate_structural=False):
    
    data = load_data(dataset_str)
    node_features = data.x
    
    if args.normalize_feat:
    
        node_features_min = node_features.min()
        node_features_max = node_features.max()
        node_features = (node_features - node_features_min)/node_features_max
        data.x = node_features
    
    yc = []
    ys = []
    yj = []
    
    if calculate_contextual:
        
        if dataset_str == "inj_cora":
            yc = data.y >> 0 & 1 # contextual outliers
        else:
            data, yc = gen_contextual_outliers(data=data,n=args.contextual_n,k=args.contextual_k)
            
        yc = yc.cpu().detach()
    
    
    if calculate_structural:
        
        if dataset_str == "inj_cora":
            ys = data.y >> 1 & 1 # structural outliers
        else:
            data, ys = gen_structural_outliers(data=data,n=args.structural_n,m=args.structural_m,p=0.2)
            
        ys = ys.cpu().detach()
        data, yj = gen_joint_structural_outliers(data=data,n=args.structural_n,m=args.structural_m)
        
    
    if args.use_combine_outlier:
        data.y = torch.logical_or(ys, yc).int()
        
    ysj = torch.logical_or(ys, yj).int()
    y = data.y.bool()    # binary labels (inlier/outlier)
    y = y.cpu().detach()
    
    edge_index = data.edge_index.cpu()
    
    num_nodes = node_features.shape[0]
    self_edges = torch.tensor([[i for i in range(num_nodes)],[i for i in range(num_nodes)]])
    edge_index = torch.cat([edge_index,self_edges],dim=1)
    data.edge_index = edge_index
    data = data.to(device)
    

    loss, loss_per_node, = train(data, y, yc, ys, yj, ysj, lr=lr, epoch=epoch_num, device=device, encoder=encoder, lambda_loss1=lambda_loss1, 
          lambda_loss2=lambda_loss2, lambda_loss3=lambda_loss3, hidden_dim=hidden_dim, sample_size=sample_size,loss_step=loss_step,
                                 real_loss=real_loss,calculate_contextual=calculate_contextual,calculate_structural=calculate_structural)

In [15]:
# generate ground truth neighbors Hv
def generate_gt_neighbor(neighbor_dict, node_embeddings, neighbor_num_list, in_dim):
    max_neighbor_num = max(neighbor_num_list)
    all_gt_neighbor_embeddings = []
    for i, embedding in enumerate(node_embeddings):
        neighbor_indexes = neighbor_dict[i]
        neighbor_embeddings = []
        for index in neighbor_indexes:
            neighbor_embeddings.append(node_embeddings[index].tolist())
        if len(neighbor_embeddings) < max_neighbor_num:
            for _ in range(max_neighbor_num - len(neighbor_embeddings)):
                neighbor_embeddings.append(torch.zeros(in_dim).tolist())
        all_gt_neighbor_embeddings.append(neighbor_embeddings)
    return all_gt_neighbor_embeddings


# Main Autoencoder structure here
class GNNStructEncoder(nn.Module):
    def __init__(self, in_dim0, in_dim, hidden_dim, layer_num, sample_size, device, neighbor_num_list, 
                 GNN_name="GIN", norm_mode="PN-SCS", norm_scale=20, lambda_loss1=0.01, lambda_loss2=0.001, lambda_loss3=0.0001):
        
        super(GNNStructEncoder, self).__init__()
        
        self.mlp0 = nn.Linear(in_dim0, hidden_dim)
        self.norm = PairNorm(norm_mode, norm_scale)
        self.out_dim = hidden_dim
        self.lambda_loss1 = lambda_loss1
        self.lambda_loss2 = lambda_loss2
        self.lambda_loss3 = lambda_loss3
        # GNN Encoder
        if GNN_name == "GIN":
            self.linear1 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv1 = GINConv(self.linear1)
            self.linear2 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv2 = GINConv(self.linear2)
        elif GNN_name == "GCN":
            self.graphconv1 = GCNConv(hidden_dim, hidden_dim)
            self.graphconv2 = GCNConv(hidden_dim, hidden_dim)
        elif GNN_name == "GAT":
            self.graphconv1 = GATConv(hidden_dim, hidden_dim)
            self.graphconv2 = GATConv(hidden_dim, hidden_dim)
        else:
            self.graphconv1 = SAGEConv(hidden_dim, hidden_dim, aggr=args.aggregator)
            # self.graphconv2 = SAGEConv(hidden_dim, hidden_dim, aggr='mean')
            
            # self.graphconv1 = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
            

        self.neighbor_num_list = neighbor_num_list
        self.tot_node = len(neighbor_num_list)

        self.gaussian_mean = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.m = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            torch.ones(sample_size, hidden_dim))
        
        self.m_batched = torch.distributions.Normal(torch.zeros(sample_size, self.tot_node, hidden_dim),
                                            torch.ones(sample_size, self.tot_node, hidden_dim))

        self.m_h = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            50* torch.ones(sample_size, hidden_dim))

        # Before MLP Gaussian Means, and std

        self.mlp_gaussian_mean = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_m = torch.distributions.Normal(torch.zeros(hidden_dim), torch.ones(hidden_dim))

        self.mlp_mean = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.mlp_sigma = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.softplus = nn.Softplus()

        self.mean_agg = SAGEConv(hidden_dim, hidden_dim, aggr=args.aggregator, normalize = False)
        # self.mean_agg = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
        self.std_agg = PNAConv(hidden_dim, hidden_dim, aggregators=["std"],scalers=["identity"], deg=neighbor_num_list)        
        self.layer1_generator = MLP_generator(hidden_dim, hidden_dim)
        
        # Decoders
        self.degree_decoder = FNN(hidden_dim, hidden_dim, 1, 4)
        self.feature_decoder = FNN(hidden_dim, hidden_dim, in_dim, 3)
        self.degree_loss_func = nn.MSELoss()
        self.feature_loss_func = nn.MSELoss()
        self.pool = mp.Pool(4)
        self.in_dim = in_dim
        self.sample_size = sample_size 
        self.init_projection = FNN(in_dim, hidden_dim, hidden_dim, 1)
        

    def forward_encoder(self, x, edge_index):
        
        # Apply graph convolution and activation, pair-norm to avoid trivial solution
        h0 = self.mlp0(x)
        l1 = self.graphconv1(h0, edge_index)
        return l1, h0
        
        

    # Sample neighbors from neighbor set, if the length of neighbor set less than sample size, then do the padding.
    def sample_neighbors(self, indexes, neighbor_dict, gt_embeddings):
        sampled_embeddings_list = []
        mark_len_list = []
        for index in indexes:
            sampled_embeddings = []
            neighbor_indexes = neighbor_dict[index]
            if len(neighbor_indexes) < self.sample_size:
                mask_len = len(neighbor_indexes)
                sample_indexes = neighbor_indexes
            else:
                sample_indexes = random.sample(neighbor_indexes, self.sample_size)
                mask_len = self.sample_size
            for index in sample_indexes:
                sampled_embeddings.append(gt_embeddings[index].tolist())
            if len(sampled_embeddings) < self.sample_size:
                for _ in range(self.sample_size - len(sampled_embeddings)):
                    sampled_embeddings.append(torch.zeros(self.out_dim).tolist())
            sampled_embeddings_list.append(sampled_embeddings)
            mark_len_list.append(mask_len)
        
        return sampled_embeddings_list, mark_len_list
    
    def reconstruction_neighbors2(self, l1, h0, edge_index):
                
        recon_loss = 0
        recon_loss_per_node = []
    
        sample_sz_per_node = [self.sample_size]* self.tot_node

        mean_neigh = self.mean_agg.forward(h0, edge_index).detach()
        std_neigh = self.std_agg(h0, edge_index).detach()
        
        
        cov_neigh = torch.bmm(std_neigh.unsqueeze(dim=-1),std_neigh.unsqueeze(dim=1))
        
        target_mean = mean_neigh
        target_cov = cov_neigh
        
        self_embedding = l1
        # self_embedding = _normalize(self_embedding)
        
        self_embedding = self_embedding.unsqueeze(0)
        self_embedding = self_embedding.repeat(self.sample_size, 1, 1)
        generated_mean = self.mlp_mean(self_embedding)
        generated_sigma = self.mlp_sigma(self_embedding)

        
        std_z = self.m_batched.sample().to(device)
        var = generated_mean + generated_sigma.exp() * std_z
        nhij = self.layer1_generator(var)
        
        generated_mean = torch.mean(nhij,dim=0)
        generated_std = torch.std(nhij,dim=0)
        generated_cov = torch.bmm(generated_std.unsqueeze(dim=-1),generated_std.unsqueeze(dim=1))/self.sample_size
        
        
        tot_nodes = l1.shape[0]
        h_dim = l1.shape[1]
        
        single_eye = torch.eye(h_dim).to(device)
        single_eye = single_eye.unsqueeze(dim=0)
        batch_eye = single_eye.repeat(tot_nodes,1,1)
        
        target_cov = target_cov + batch_eye
        generated_cov = generated_cov + batch_eye

        
        det_target_cov = torch.linalg.det(target_cov) 
        det_generated_cov = torch.linalg.det(generated_cov) 
        trace_mat = torch.matmul(torch.inverse(generated_cov),target_cov)
             
        
        x = torch.bmm(torch.unsqueeze(generated_mean - target_mean,dim=1),torch.inverse(generated_cov))
        y = torch.unsqueeze(generated_mean - target_mean,dim=-1)
        z = torch.bmm(x,y).squeeze()
        
        KL_loss = 0.5 * (torch.log(det_target_cov / det_generated_cov) - h_dim  + trace_mat.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) + z)
        
        recon_loss = torch.mean(KL_loss)
        recon_loss_per_node = KL_loss
        
        
        return recon_loss, recon_loss_per_node
    

    def reconstruction_neighbors(self, FNN_generator, neighbor_indexes, neighbor_dict, from_layer, to_layer, device):
        
        
        local_index_loss = 0
        local_index_loss_per_node = []
        sampled_embeddings_list, mark_len_list = self.sample_neighbors(neighbor_indexes, neighbor_dict, to_layer)
        for i, neighbor_embeddings1 in enumerate(sampled_embeddings_list):
            # Generating h^k_v, reparameterization trick
            
            # print(len(neighbor_embeddings1))
            
            
            index = neighbor_indexes[i]
            mask_len1 = mark_len_list[i]
            mean = from_layer[index].repeat(self.sample_size, 1)
            mean = self.mlp_mean(mean)
            sigma = from_layer[index].repeat(self.sample_size, 1)
            sigma = self.mlp_sigma(sigma)
            std_z = self.m.sample().to(device)
            var = mean + sigma.exp() * std_z
            nhij = FNN_generator(var)
            
            generated_neighbors = nhij
            sum_neighbor_norm = 0
            
            for indexi, generated_neighbor in enumerate(generated_neighbors):
                sum_neighbor_norm += torch.norm(generated_neighbor) / math.sqrt(self.out_dim)
            generated_neighbors = torch.unsqueeze(generated_neighbors, dim=0).to(device)
            target_neighbors = torch.unsqueeze(torch.FloatTensor(neighbor_embeddings1), dim=0).to(device)
            
            if args.neigh_loss == "KL":
                KL_loss = KL_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += KL_loss
                local_index_loss_per_node.append(KL_loss)
            
            else:
                W2_loss = W2_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += W2_loss
                local_index_loss_per_node.append(W2_loss)
            
            
        local_index_loss_per_node = torch.stack(local_index_loss_per_node)
        return local_index_loss, local_index_loss_per_node
    

    def neighbor_decoder(self, gij, ground_truth_degree_matrix, h0, neighbor_dict, device, h, edge_index):
        
        # Degree decoder below:
        tot_nodes = gij.shape[0]
        degree_logits = self.degree_decoding(gij)
        ground_truth_degree_matrix = torch.unsqueeze(ground_truth_degree_matrix, dim=1)
        degree_loss = self.degree_loss_func(degree_logits, ground_truth_degree_matrix.float())
        degree_loss_per_node = (degree_logits-ground_truth_degree_matrix).pow(2)
        _, degree_masks = torch.max(degree_logits.data, dim=1)
        h_loss = 0
        feature_loss = 0
        # layer 1
        loss_list = []
        loss_list_per_node = []
        feature_loss_list = []
        # Sample multiple times to remove noise
        for _ in range(3):
            local_index_loss_sum = 0
            local_index_loss_sum_per_node = []
            indexes = []
            h0_prime = self.feature_decoder(gij)
            feature_losses = self.feature_loss_func(h0, h0_prime)
            feature_losses_per_node = (h0-h0_prime).pow(2).mean(1)
            feature_loss_list.append(feature_losses_per_node)
            

            
            local_index_loss, local_index_loss_per_node = self.reconstruction_neighbors2(gij,h0,edge_index)
            
            loss_list.append(local_index_loss)
            loss_list_per_node.append(local_index_loss_per_node)
            
        loss_list = torch.stack(loss_list)
        h_loss += torch.mean(loss_list)
        
        loss_list_per_node = torch.stack(loss_list_per_node)
        h_loss_per_node = torch.mean(loss_list_per_node,dim=0)
        
        feature_loss_per_node = torch.mean(torch.stack(feature_loss_list),dim=0)
        feature_loss += torch.mean(torch.stack(feature_loss_list))
                
        h_loss_per_node = h_loss_per_node.reshape(tot_nodes,1)
        degree_loss_per_node = degree_loss_per_node.reshape(tot_nodes,1)
        feature_loss_per_node = feature_loss_per_node.reshape(tot_nodes,1)
        
        loss = self.lambda_loss1 * h_loss + degree_loss * self.lambda_loss3 + self.lambda_loss2 * feature_loss
        loss_per_node = self.lambda_loss1 * h_loss_per_node + degree_loss_per_node * self.lambda_loss3 + self.lambda_loss2 * feature_loss_per_node
        

        return loss,loss_per_node,h_loss_per_node,degree_loss_per_node,feature_loss_per_node

    def degree_decoding(self, node_embeddings):
        degree_logits = F.relu(self.degree_decoder(node_embeddings))
        return degree_logits

    def forward(self, edge_index, x, ground_truth_degree_matrix, neighbor_dict, device):
        
        # Generate GNN encodings
        l1, h0 = self.forward_encoder(x, edge_index)
        loss, loss_per_node,h_loss,degree_loss,feature_loss = self.neighbor_decoder(l1, ground_truth_degree_matrix, h0, neighbor_dict, device, x, edge_index)
        
        return loss, loss_per_node,h_loss,degree_loss,feature_loss


# Execution

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="books")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.5) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=True) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--aggregator', type=str, default="mean")



# args = parser.parse_args()
args = parser.parse_args([])



print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  books lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.5 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: True h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  1%|          | 4/500 [00:00<00:28, 17.33it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  47.744090441932165
Dataset Name:  books , AUC Score (contextual):  25.81145706145706
Dataset Name:  books , AUC Score (structural):  49.1627808393387
Dataset Name:  books , AUC Score (joint-type):  39.37474562474562
Dataset Name:  books , AUC Score (structure type):  47.52944361716529
Dataset Name:  books  Best AUC Score(benchmark/combined):  47.744090441932165
Dataset Name:  books  Best AUC Score (contextual):  25.81145706145706
Dataset Name:  books  Best AUC Score (structural):  49.1627808393387
Dataset Name:  books  Best AUC Score (joint-type):  39.37474562474562
Dataset Name:  books  Best AUC Score (structure type):  47.52944361716529
Dataset Name:  books , AUC Score(benchmark/combined):  48.4994861253854
Dataset Name:  books , AUC Score (contextual):  21.420940170940167
Dataset Name:  books , AUC Score (structural):  49.698495125052986
Dataset Name:  books , AUC Score (joint-type):  41.08160358160358
Dataset Name:  books , AUC

  1%|▏         | 7/500 [00:00<00:24, 19.98it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  48.65878725590955
Dataset Name:  books , AUC Score (contextual):  23.758648758648757
Dataset Name:  books , AUC Score (structural):  50.0784230606189
Dataset Name:  books , AUC Score (joint-type):  36.950549450549445
Dataset Name:  books , AUC Score (structure type):  47.894048102522454
Dataset Name:  books  Best AUC Score(benchmark/combined):  49.269013360739976
Dataset Name:  books  Best AUC Score (contextual):  25.81145706145706
Dataset Name:  books  Best AUC Score (structural):  50.82291225095379
Dataset Name:  books  Best AUC Score (joint-type):  36.950549450549445
Dataset Name:  books  Best AUC Score (structure type):  49.2306303867154
Dataset Name:  books , AUC Score(benchmark/combined):  49.40390544707092
Dataset Name:  books , AUC Score (contextual):  23.58058608058608
Dataset Name:  books , AUC Score (structural):  50.48060618906316
Dataset Name:  books , AUC Score (joint-type):  39.46123321123321
Dataset Name:  books , A

...

Dataset Name:  books , AUC Score(benchmark/combined):  62.78006166495375
Dataset Name:  books , AUC Score (contextual):  64.27299552299553
Dataset Name:  books , AUC Score (structural):  47.19266638406104
Dataset Name:  books , AUC Score (joint-type):  41.36141636141636
Dataset Name:  books , AUC Score (structure type):  45.960922341049596
Dataset Name:  books  Best AUC Score(benchmark/combined):  66.76002055498459
Dataset Name:  books  Best AUC Score (contextual):  68.77543752543752
Dataset Name:  books  Best AUC Score (structural):  50.99830436625689
Dataset Name:  books  Best AUC Score (joint-type):  41.36141636141636
Dataset Name:  books  Best AUC Score (structure type):  49.9923288660259
Dataset Name:  books , AUC Score(benchmark/combined):  61.59558067831448
Dataset Name:  books , AUC Score (contextual):  63.91178266178266
Dataset Name:  books , AUC Score (structural):  45.51080966511233
Dataset Name:  books , AUC Score (joint-type):  41.005291005291014
Dataset Name:  books , AUC

 99%|█████████▉| 496/500 [00:21<00:00, 22.34it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  61.41829393627955
Dataset Name:  books , AUC Score (contextual):  64.01862026862027
Dataset Name:  books , AUC Score (structural):  45.46100042390844
Dataset Name:  books , AUC Score (joint-type):  41.31054131054131
Dataset Name:  books , AUC Score (structure type):  44.52235909931863
Dataset Name:  books  Best AUC Score(benchmark/combined):  66.76002055498459
Dataset Name:  books  Best AUC Score (contextual):  68.77543752543752
Dataset Name:  books  Best AUC Score (structural):  50.99830436625689
Dataset Name:  books  Best AUC Score (joint-type):  41.31054131054131
Dataset Name:  books  Best AUC Score (structure type):  49.9923288660259
Dataset Name:  books , AUC Score(benchmark/combined):  62.38694758478932
Dataset Name:  books , AUC Score (contextual):  65.48890923890923
Dataset Name:  books , AUC Score (structural):  47.472445951674445
Dataset Name:  books , AUC Score (joint-type):  41.325803825803824
Dataset Name:  books , AUC

100%|██████████| 500/500 [00:21<00:00, 23.28it/s]


Dataset Name:  books , AUC Score(benchmark/combined):  61.228160328879746
Dataset Name:  books , AUC Score (contextual):  63.67775742775743
Dataset Name:  books , AUC Score (structural):  46.88003391267486
Dataset Name:  books , AUC Score (joint-type):  38.35470085470085
Dataset Name:  books , AUC Score (structure type):  45.22629845223591
Dataset Name:  books  Best AUC Score(benchmark/combined):  66.76002055498459
Dataset Name:  books  Best AUC Score (contextual):  68.77543752543752
Dataset Name:  books  Best AUC Score (structural):  50.99830436625689
Dataset Name:  books  Best AUC Score (joint-type):  38.35470085470085
Dataset Name:  books  Best AUC Score (structure type):  49.9923288660259
Dataset Name:  books , AUC Score(benchmark/combined):  61.10483042137719
Dataset Name:  books , AUC Score (contextual):  64.68508343508344
Dataset Name:  books , AUC Score (structural):  46.11593895718524
Dataset Name:  books , AUC Score (joint-type):  40.27777777777777
Dataset Name:  books , AUC 

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="books")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.5) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=True) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--aggregator', type=str, default="mean")



# args = parser.parse_args()
args = parser.parse_args([])



print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  books lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.5 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: True h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 2/500 [00:00<00:29, 17.14it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  46.065005138746145
Dataset Name:  books , AUC Score (contextual):  47.502035002035
Dataset Name:  books , AUC Score (structural):  52.51801610852056
Dataset Name:  books , AUC Score (joint-type):  46.82539682539683
Dataset Name:  books , AUC Score (structure type):  51.563503962304566
Dataset Name:  books  Best AUC Score(benchmark/combined):  46.065005138746145
Dataset Name:  books  Best AUC Score (contextual):  47.502035002035
Dataset Name:  books  Best AUC Score (structural):  52.51801610852056
Dataset Name:  books  Best AUC Score (joint-type):  46.82539682539683
Dataset Name:  books  Best AUC Score (structure type):  51.563503962304566
Dataset Name:  books , AUC Score(benchmark/combined):  47.02980472764646
Dataset Name:  books , AUC Score (contextual):  46.91442816442816
Dataset Name:  books , AUC Score (structural):  54.76896990250106
Dataset Name:  books , AUC Score (joint-type):  47.77930402930403
Dataset Name:  books , AUC 

  2%|▏         | 8/500 [00:00<00:23, 21.30it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  44.94090441932169
Dataset Name:  books , AUC Score (contextual):  41.73789173789174
Dataset Name:  books , AUC Score (structural):  52.70241627808393
Dataset Name:  books , AUC Score (joint-type):  48.618742368742375
Dataset Name:  books , AUC Score (structure type):  52.03335832083958
Dataset Name:  books  Best AUC Score(benchmark/combined):  47.02980472764646
Dataset Name:  books  Best AUC Score (contextual):  47.502035002035
Dataset Name:  books  Best AUC Score (structural):  54.76896990250106
Dataset Name:  books  Best AUC Score (joint-type):  48.618742368742375
Dataset Name:  books  Best AUC Score (structure type):  53.626311844077954
Dataset Name:  books , AUC Score(benchmark/combined):  45.96736896197328
Dataset Name:  books , AUC Score (contextual):  43.03266178266178
Dataset Name:  books , AUC Score (structural):  53.6715769393811
Dataset Name:  books , AUC Score (joint-type):  45.20502645502646
Dataset Name:  books , AUC 

...

Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , AUC Score (structure type):  46.40581494966802
Dataset Name:  books  Best AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books  Best AUC Score (contextual):  49.71509971509972
Dataset Name:  books  Best AUC Score (structural):  54.76896990250106
Dataset Name:  books  Best AUC Score (joint-type):  48.048941798941804
Dataset Name:  books  Best AUC Score (structure type):  53.626311844077954
Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , 

 99%|█████████▉| 494/500 [00:21<00:00, 22.70it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , AUC Score (structure type):  46.40581494966802
Dataset Name:  books  Best AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books  Best AUC Score (contextual):  49.71509971509972
Dataset Name:  books  Best AUC Score (structural):  54.76896990250106
Dataset Name:  books  Best AUC Score (joint-type):  48.048941798941804
Dataset Name:  books  Best AUC Score (structure type):  53.626311844077954
Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , 

100%|██████████| 500/500 [00:21<00:00, 22.79it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , AUC Score (structure type):  46.40581494966802
Dataset Name:  books  Best AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books  Best AUC Score (contextual):  49.71509971509972
Dataset Name:  books  Best AUC Score (structural):  54.76896990250106
Dataset Name:  books  Best AUC Score (joint-type):  48.048941798941804
Dataset Name:  books  Best AUC Score (structure type):  53.626311844077954
Dataset Name:  books , AUC Score(benchmark/combined):  63.08067831449126
Dataset Name:  books , AUC Score (contextual):  41.62087912087912
Dataset Name:  books , AUC Score (structural):  46.13819415006359
Dataset Name:  books , AUC Score (joint-type):  48.048941798941804
Dataset Name:  books , 

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="books")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.5) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=True) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--aggregator', type=str, default="mean")



# args = parser.parse_args()
args = parser.parse_args([])



print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  books lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.5 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: True h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 2/500 [00:00<00:33, 14.77it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  47.37024665981501
Dataset Name:  books , AUC Score (contextual):  55.769230769230774
Dataset Name:  books , AUC Score (structural):  49.74406528189911
Dataset Name:  books , AUC Score (joint-type):  60.93813593813594
Dataset Name:  books , AUC Score (structure type):  51.703166274005866
Dataset Name:  books  Best AUC Score(benchmark/combined):  47.37024665981501
Dataset Name:  books  Best AUC Score (contextual):  55.769230769230774
Dataset Name:  books  Best AUC Score (structural):  49.74406528189911
Dataset Name:  books  Best AUC Score (joint-type):  60.93813593813594
Dataset Name:  books  Best AUC Score (structure type):  51.703166274005866
Dataset Name:  books , AUC Score(benchmark/combined):  47.71582733812949
Dataset Name:  books , AUC Score (contextual):  59.09137159137159
Dataset Name:  books , AUC Score (structural):  50.980818143281056
Dataset Name:  books , AUC Score (joint-type):  59.803622303622305
Dataset Name:  books 

  2%|▏         | 8/500 [00:00<00:23, 20.69it/s]

Dataset Name:  books , AUC Score (joint-type):  58.508852258852265
Dataset Name:  books , AUC Score (structure type):  51.8539837224245
Dataset Name:  books  Best AUC Score(benchmark/combined):  47.71582733812949
Dataset Name:  books  Best AUC Score (contextual):  59.09137159137159
Dataset Name:  books  Best AUC Score (structural):  50.980818143281056
Dataset Name:  books  Best AUC Score (joint-type):  58.508852258852265
Dataset Name:  books  Best AUC Score (structure type):  52.54560219890054
Dataset Name:  books , AUC Score(benchmark/combined):  45.68730729701953
Dataset Name:  books , AUC Score (contextual):  55.78958078958078
Dataset Name:  books , AUC Score (structural):  50.71216617210683
Dataset Name:  books , AUC Score (joint-type):  59.85704110704111
Dataset Name:  books , AUC Score (structure type):  52.32874634111515
Dataset Name:  books  Best AUC Score(benchmark/combined):  47.71582733812949
Dataset Name:  books  Best AUC Score (contextual):  59.09137159137159
Dataset Name:

...

Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books , AUC Score (structure type):  50.65369101163704
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books  Best AUC Score (contextual):  59.289784289784286
Dataset Name:  books  Best AUC Score (structural):  53.03465451462485
Dataset Name:  books  Best AUC Score (joint-type):  43.727106227106226
Dataset Name:  books  Best AUC Score (structure type):  52.9819911472835
Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books ,

 99%|█████████▉| 497/500 [00:22<00:00, 22.30it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books , AUC Score (structure type):  50.65369101163704
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books  Best AUC Score (contextual):  59.289784289784286
Dataset Name:  books  Best AUC Score (structural):  53.03465451462485
Dataset Name:  books  Best AUC Score (joint-type):  43.727106227106226
Dataset Name:  books  Best AUC Score (structure type):  52.9819911472835
Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books ,

100%|██████████| 500/500 [00:22<00:00, 21.94it/s]


Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books , AUC Score (structure type):  50.65369101163704
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books  Best AUC Score (contextual):  59.289784289784286
Dataset Name:  books  Best AUC Score (structural):  53.03465451462485
Dataset Name:  books  Best AUC Score (joint-type):  43.727106227106226
Dataset Name:  books  Best AUC Score (structure type):  52.9819911472835
Dataset Name:  books , AUC Score(benchmark/combined):  64.52980472764645
Dataset Name:  books , AUC Score (contextual):  44.057794057794055
Dataset Name:  books , AUC Score (structural):  52.08298007630352
Dataset Name:  books , AUC Score (joint-type):  43.727106227106226
Dataset Name:  books ,

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="books")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.5) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=True) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--aggregator', type=str, default="mean")



# args = parser.parse_args()
args = parser.parse_args([])



print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  books lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.5 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: True h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 2/500 [00:00<00:32, 15.46it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  42.747944501541625
Dataset Name:  books , AUC Score (contextual):  58.18325193325193
Dataset Name:  books , AUC Score (structural):  53.232831708351
Dataset Name:  books , AUC Score (joint-type):  57.51170126170126
Dataset Name:  books , AUC Score (structure type):  54.118496457741074
Dataset Name:  books  Best AUC Score(benchmark/combined):  42.747944501541625
Dataset Name:  books  Best AUC Score (contextual):  58.18325193325193
Dataset Name:  books  Best AUC Score (structural):  53.232831708351
Dataset Name:  books  Best AUC Score (joint-type):  57.51170126170126
Dataset Name:  books  Best AUC Score (structure type):  54.118496457741074
Dataset Name:  books , AUC Score(benchmark/combined):  43.819373072970194
Dataset Name:  books , AUC Score (contextual):  53.37810337810337
Dataset Name:  books , AUC Score (structural):  53.33774904620602
Dataset Name:  books , AUC Score (joint-type):  61.69871794871796
Dataset Name:  books , AUC

  2%|▏         | 8/500 [00:00<00:24, 20.30it/s]

Dataset Name:  books , AUC Score (contextual):  56.982600732600744
Dataset Name:  books , AUC Score (structural):  53.46863077575244
Dataset Name:  books , AUC Score (joint-type):  59.238909238909244
Dataset Name:  books , AUC Score (structure type):  54.64284102702946
Dataset Name:  books  Best AUC Score(benchmark/combined):  43.819373072970194
Dataset Name:  books  Best AUC Score (contextual):  58.18325193325193
Dataset Name:  books  Best AUC Score (structural):  54.175498092412035
Dataset Name:  books  Best AUC Score (joint-type):  59.238909238909244
Dataset Name:  books  Best AUC Score (structure type):  54.93163665899553
Dataset Name:  books , AUC Score(benchmark/combined):  43.98766700924974
Dataset Name:  books , AUC Score (contextual):  55.63949938949939
Dataset Name:  books , AUC Score (structural):  52.754345061466715
Dataset Name:  books , AUC Score (joint-type):  57.9008954008954
Dataset Name:  books , AUC Score (structure type):  53.80262623527818
Dataset Name:  books  Bes

...

Dataset Name:  books , AUC Score(benchmark/combined):  54.17523124357656
Dataset Name:  books , AUC Score (contextual):  53.67572242572243
Dataset Name:  books , AUC Score (structural):  53.52002967359051
Dataset Name:  books , AUC Score (joint-type):  53.82834757834758
Dataset Name:  books , AUC Score (structure type):  53.635215017372865
Dataset Name:  books  Best AUC Score(benchmark/combined):  54.646711202466605
Dataset Name:  books  Best AUC Score (contextual):  60.20807895807895
Dataset Name:  books  Best AUC Score (structural):  54.903560830860535
Dataset Name:  books  Best AUC Score (joint-type):  53.82834757834758
Dataset Name:  books  Best AUC Score (structure type):  55.6978475700555
Dataset Name:  books , AUC Score(benchmark/combined):  52.20966084275436
Dataset Name:  books , AUC Score (contextual):  56.52218152218151
Dataset Name:  books , AUC Score (structural):  53.06803730394235
Dataset Name:  books , AUC Score (joint-type):  51.836589336589334
Dataset Name:  books , A

 99%|█████████▉| 497/500 [00:22<00:00, 20.85it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  52.73124357656731
Dataset Name:  books , AUC Score (contextual):  56.1965811965812
Dataset Name:  books , AUC Score (structural):  53.16129715981348
Dataset Name:  books , AUC Score (joint-type):  52.182539682539684
Dataset Name:  books , AUC Score (structure type):  53.0774784531384
Dataset Name:  books  Best AUC Score(benchmark/combined):  54.646711202466605
Dataset Name:  books  Best AUC Score (contextual):  60.20807895807895
Dataset Name:  books  Best AUC Score (structural):  54.903560830860535
Dataset Name:  books  Best AUC Score (joint-type):  52.182539682539684
Dataset Name:  books  Best AUC Score (structure type):  55.6978475700555
Dataset Name:  books , AUC Score(benchmark/combined):  52.756937307297015
Dataset Name:  books , AUC Score (contextual):  55.8048433048433
Dataset Name:  books , AUC Score (structural):  53.15864773208987
Dataset Name:  books , AUC Score (joint-type):  51.23371998371999
Dataset Name:  books , AUC

100%|██████████| 500/500 [00:22<00:00, 22.05it/s]


Dataset Name:  books , AUC Score(benchmark/combined):  51.35020554984584
Dataset Name:  books , AUC Score (contextual):  57.763532763532766
Dataset Name:  books , AUC Score (structural):  53.50466299279355
Dataset Name:  books , AUC Score (joint-type):  51.170126170126174
Dataset Name:  books , AUC Score (structure type):  53.190289246875146
Dataset Name:  books  Best AUC Score(benchmark/combined):  54.646711202466605
Dataset Name:  books  Best AUC Score (contextual):  60.20807895807895
Dataset Name:  books  Best AUC Score (structural):  54.903560830860535
Dataset Name:  books  Best AUC Score (joint-type):  51.170126170126174
Dataset Name:  books  Best AUC Score (structure type):  55.6978475700555
Dataset Name:  books , AUC Score(benchmark/combined):  52.73124357656731
Dataset Name:  books , AUC Score (contextual):  56.1965811965812
Dataset Name:  books , AUC Score (structural):  53.16129715981348
Dataset Name:  books , AUC Score (joint-type):  52.182539682539684
Dataset Name:  books ,

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="books")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.5) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=100)
parser.add_argument('--real_loss', type=bool, default=True) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.0) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=14)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=14)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=True)
parser.add_argument('--aggregator', type=str, default="mean")



# args = parser.parse_args()
args = parser.parse_args([])



print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  books lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.5 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 100 real_loss: True h_loss_weight: 1.0 feature_loss_weight 2.0 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 2/500 [00:00<00:31, 15.99it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  42.31243576567317
Dataset Name:  books , AUC Score (contextual):  51.23626373626373
Dataset Name:  books , AUC Score (structural):  56.60555320050869
Dataset Name:  books , AUC Score (joint-type):  50.13481888481889
Dataset Name:  books , AUC Score (structure type):  55.39424200379728
Dataset Name:  books  Best AUC Score(benchmark/combined):  42.31243576567317
Dataset Name:  books  Best AUC Score (contextual):  51.23626373626373
Dataset Name:  books  Best AUC Score (structural):  56.60555320050869
Dataset Name:  books  Best AUC Score (joint-type):  50.13481888481889
Dataset Name:  books  Best AUC Score (structure type):  55.39424200379728
Dataset Name:  books , AUC Score(benchmark/combined):  43.96968139773895
Dataset Name:  books , AUC Score (contextual):  50.643569393569386
Dataset Name:  books , AUC Score (structural):  55.800127172530736
Dataset Name:  books , AUC Score (joint-type):  49.53194953194953
Dataset Name:  books , AU

  2%|▏         | 8/500 [00:00<00:21, 22.96it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  46.07656731757451
Dataset Name:  books , AUC Score (contextual):  47.1052096052096
Dataset Name:  books , AUC Score (structural):  57.110004239084354
Dataset Name:  books , AUC Score (joint-type):  52.7981277981278
Dataset Name:  books , AUC Score (structure type):  56.164195998247415
Dataset Name:  books  Best AUC Score(benchmark/combined):  46.07656731757451
Dataset Name:  books  Best AUC Score (contextual):  51.23626373626373
Dataset Name:  books  Best AUC Score (structural):  57.110004239084354
Dataset Name:  books  Best AUC Score (joint-type):  52.7981277981278
Dataset Name:  books  Best AUC Score (structure type):  56.164195998247415
Dataset Name:  books , AUC Score(benchmark/combined):  46.26541623843782
Dataset Name:  books , AUC Score (contextual):  46.72364672364672
Dataset Name:  books , AUC Score (structural):  57.11954217888937
Dataset Name:  books , AUC Score (joint-type):  52.07061457061457
Dataset Name:  books , AUC

...

Dataset Name:  books , AUC Score(benchmark/combined):  62.47430626927029
Dataset Name:  books , AUC Score (contextual):  49.72527472527472
Dataset Name:  books , AUC Score (structural):  55.102797795676125
Dataset Name:  books , AUC Score (joint-type):  44.190069190069195
Dataset Name:  books , AUC Score (structure type):  52.96936614575727
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.21891058581706
Dataset Name:  books  Best AUC Score (contextual):  60.027472527472526
Dataset Name:  books  Best AUC Score (structural):  58.306485799067396
Dataset Name:  books  Best AUC Score (joint-type):  44.190069190069195
Dataset Name:  books  Best AUC Score (structure type):  56.505586388199205
Dataset Name:  books , AUC Score(benchmark/combined):  62.20966084275437
Dataset Name:  books , AUC Score (contextual):  52.57427757427757
Dataset Name:  books , AUC Score (structural):  55.45252225519288
Dataset Name:  books , AUC Score (joint-type):  44.31725681725682
Dataset Name:  books 

 99%|█████████▉| 494/500 [00:21<00:00, 22.69it/s]

Dataset Name:  books , AUC Score(benchmark/combined):  62.66700924974307
Dataset Name:  books , AUC Score (contextual):  53.14916564916565
Dataset Name:  books , AUC Score (structural):  55.93259855871131
Dataset Name:  books , AUC Score (joint-type):  42.95889295889296
Dataset Name:  books , AUC Score (structure type):  53.52070249744414
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.21891058581706
Dataset Name:  books  Best AUC Score (contextual):  60.027472527472526
Dataset Name:  books  Best AUC Score (structural):  58.306485799067396
Dataset Name:  books  Best AUC Score (joint-type):  42.95889295889296
Dataset Name:  books  Best AUC Score (structure type):  56.505586388199205
Dataset Name:  books , AUC Score(benchmark/combined):  62.63360739979444
Dataset Name:  books , AUC Score (contextual):  50.41208791208791
Dataset Name:  books , AUC Score (structural):  55.97604917337855
Dataset Name:  books , AUC Score (joint-type):  42.765567765567766
Dataset Name:  books , 

100%|██████████| 500/500 [00:21<00:00, 23.38it/s]


Dataset Name:  books , AUC Score(benchmark/combined):  61.22045220966085
Dataset Name:  books , AUC Score (contextual):  49.26739926739927
Dataset Name:  books , AUC Score (structural):  55.27766002543451
Dataset Name:  books , AUC Score (joint-type):  44.20533170533171
Dataset Name:  books , AUC Score (structure type):  53.108113042208274
Dataset Name:  books  Best AUC Score(benchmark/combined):  64.21891058581706
Dataset Name:  books  Best AUC Score (contextual):  60.027472527472526
Dataset Name:  books  Best AUC Score (structural):  58.306485799067396
Dataset Name:  books  Best AUC Score (joint-type):  44.20533170533171
Dataset Name:  books  Best AUC Score (structure type):  56.505586388199205
Dataset Name:  books , AUC Score(benchmark/combined):  61.80626927029805
Dataset Name:  books , AUC Score (contextual):  51.35327635327636
Dataset Name:  books , AUC Score (structural):  55.38257736328953
Dataset Name:  books , AUC Score (joint-type):  44.40883190883191
Dataset Name:  books , 